## Mini-project II

The details for the miniproject from Week 2

In this miniproject, we will combine and practice topics that we have learned in previous two weeks:

    APIs
    Databases (SQL)
    Pandas
    Processing special data types in Python

We will work with these APIs:

    Foursquare - we have already come across this one
    Yelp - this API offers similar services as Foursquare.
    (Stretch) Google Places API - this google api offers similar service as well.

The main goal of the mini-project is to build the database of restaurants, bars and various points of interest (POIs) in the area of your choice and find out which API has better coverage in the selected area. The APIs have limited number of requests for free, so start with the smaller area. The project consists of following tasks:

    pull the data about various POIs in the area through API. (Search Yelp for companiees that are in the area using these instructions). If you run out of requests for any of the APIs, don't worry, it's ok to use only sample data you already have or the POIs from the Yelp API. It's approach and process that counts, not the actual number of places we were able to get.
    create own SQLite database and store the data about the POIs. Think about what will be the best structure of the database. We've created sqlite3 databases before in the activity Pandas with SQL.
    compare the results using SQL or Pandas (it's up to you:)) and see which API has a better coverage of the area.
    choose the top 10 POIs based on the popularity (number of reviews or average rating) (Yelp, Foursquare).
    (Stretch) By implementing travelling salesman problem (TSP), how much time would it take to visit all of these places? (Directions API from google will be helpful here). We will have to find travel time between all places (top 10). We can use ortools from Google to effectively implement TSP. These tools are very powerful and easy to install.

We have a lot of work so let's start right away. Enjoy!!

Note

Some APIs from Google aren't for free anymore but each account has 200$ credit every month. Therefore we are able to use these services for learning purposes for free.

# SUDO code:
choice: Taipei, Taiwan
Yelp, Foursquare
0.1- successfull connection and single querry with foursquare and yelp api

1- What data do we look restaurants, bars and various points of interest (POIs) in the area of your choice?
* choose the top 10 POIs based on the popularity (number of reviews or average rating) (Yelp, Foursquare)
* have to deal with foursquare and yelp api

1.1 - we need to determine which columns are releveant

with the results:
* create a db ERD (super important)
* create the tables
* export our tables in the db

2- which API has better coverage in the selected area?
* define better
stretch (using google api)
* ow much time would it take to visit all of these places? (Directions API from google will be helpful here)


# tomorrow (sept 25) todo
clean the code, gether more data ,present the code, move the sql creation tables, create an outline + presentation

In [1]:
#python
import os
import pandas as pd
from pandas import json_normalize
import numpy as np
import foursquare
import json
import math
import types
from IPython.display import JSON

import sqlite3
from sqlite3 import Error



#user defined functions
import custom_functions as cf

  <script async defer src="https://maps.googleapis.com/maps/api/js?key=YOUR_API_KEY&callback=initMap"
  type="text/javascript"></script>

In [2]:
#KEYS
foursquare_id = os.environ["FOURSQUARE_PUBLIC_KEY"] #limit 10 000 with credit card
foursquare_secret = os.environ["FOURSQUARE_SECRET_KEY"]

yelp_client_id = os.environ['YELP_CLIENT_ID'] #limit 5000
yelp_api_key = os.environ['YELP_SECRET_KEY']

google_key=os.environ['R_DB_GOOGLE_PUBLIC_KEY'] 


## FOURSQUARE API

In [3]:
import foursquare

In [4]:
# Construct the client object
client = foursquare.Foursquare(client_id=foursquare_id, client_secret=foursquare_secret)


### single query for each categories

In [5]:
# BUILDING AND RETRIEVING THE VENUE SEARCH URL
ll = '25.033964,121.564468'#25.0330° N, 121.5654° E TAIWAN 101 tower
limit = 50 # default 20 max 50
radius = 25000 # 25km max 40000
#Categories https://developer.foursquare.com/docs/build-with-foursquare/categories/
fsq_cat = {'restaurants': '4d4b7105d754a06374d81259', 
           'bars_general': '4bf58dd8d48988d116941735', 
           'landmarks':'4bf58dd8d48988d12d941735'}
fsq_search_result = dict()
for fsq_cat,fsq_cat_code in fsq_cat.items():    
    params_vals ={
        'categoryId':fsq_cat_code, 
        'll':ll,
        'radius':radius,
        'limit':limit}

    fsq_search_result[fsq_cat] = client.venues.search(params=params_vals)

In [6]:
# fsq_search_result['restaurants'] # The dictionary works but will need to be unpacked
# print(type(fsq_search_result))
display = JSON(fsq_search_result)
display

<IPython.core.display.JSON object>

### to get the ratings we need venue details

In [7]:
#dictio = fsq_search_result
def fsq_venue_search_to_details(dictio):
    '''take the nested dictionary of foursquare search API and returns a dictionary with only name : id.
    This allows us to '''
    fsq_name_id = dict()
    for category in dictio.keys():
        for iterator in range(len(dictio[category]['venues'])):
            key = dictio[category]['venues'][iterator]['name']
            fsq_name_id[key] = dictio[category]['venues'][iterator]['id']
    return(fsq_name_id)

In [8]:
search_to_id = fsq_venue_search_to_details(fsq_search_result)

In [9]:
display = JSON(search_to_id )
display

<IPython.core.display.JSON object>

### BUILDING AND RETRIEVING THE VENUE ID URL

In [10]:
# BUILDING AND RETRIEVING THE VENUE ID URL
fsq_venue_ID_result = dict()
for venue_name,venue_id in search_to_id.items():    
    params_vals ={
        'VENUE_ID':venue_id}
 

    fsq_venue_ID_result[venue_name] = client.venues(VENUE_ID=venue_id)

In [11]:
display = JSON(fsq_venue_ID_result)
display

<IPython.core.display.JSON object>

In [12]:
fsq_venue_ID_result['Chiang Kai-Shek Memorial Hall (中正紀念堂)']['venue']['location']['lng']

121.52183532714842

In [13]:
for i in range(len(fsq_venue_ID_result['Chiang Kai-Shek Memorial Hall (中正紀念堂)']['venue']['categories'])):
    print(fsq_venue_ID_result['Chiang Kai-Shek Memorial Hall (中正紀念堂)']['venue']['categories'][0]['name'])


Monument / Landmark
Monument / Landmark
Monument / Landmark


In [14]:
def fsq_venue_data_Panda(dictio):
    
    f4sq_ready_data = {
    'name':[],
    'api':'Foursquare API',
    'api_url':[],
    'location_url':[],
    'rating':[],
    'rating_count':[],
    'category':[],
    'location_address':[],
    'lat_lon':[]    
}
    
    for venue_name in dictio.keys():
        
        short_path = dictio[venue_name]['venue']
        # no need to check since it is our key 
        f4sq_ready_data['name'].append(venue_name)
        
        f4sq_ready_data['api_url'].append(short_path.get('shortUrl',np.NaN))
        f4sq_ready_data['location_url'].append(short_path.get('url',np.NaN))
        f4sq_ready_data['rating'].append(short_path.get('rating',np.NaN))
        f4sq_ready_data['rating_count'].append(short_path.get('ratingSignals',np.NaN))
        
        #categories
        categories_names = list()
        for i in range(len(short_path['categories'])):
            categories_names.append(short_path['categories'][i]['name'])
        f4sq_ready_data['category'].append(categories_names)

        f4sq_ready_data['location_address'].append(short_path['location'].get('address',np.NaN))
        
        gps =  short_path['location'].get('lat',np.NaN) + short_path['location'].get('lng',np.NaN)
        # a single np.NaN voids any numbers present
        f4sq_ready_data['lat_lon'].append(gps)
    return f4sq_ready_data

In [15]:
f4s_r = fsq_venue_data_Panda(fsq_venue_ID_result)

In [16]:
f4sqr_SQL_DATA = pd.DataFrame(f4s_r)
f4sqr_SQL_DATA.head() 

,name,api,api_url,location_url,rating,rating_count,category,location_address,lat_lon
0,Taipei 101 Food Court (台北101美食街),Foursquare API,http://4sq.com/hS3Lgj,http://www.taipei-101.com.tw/food-search.aspx?...,7.6,164.0,[Food Court],信義路五段7號,146.597914
1,Din Tai Fung (鼎泰豐),Foursquare API,http://4sq.com/9LUPOF,https://www.dintaifung.com.tw,9.1,351.0,"[Dumpling Restaurant, Chinese Restaurant, Taiw...",忠孝東路三段300號B2,146.584690
2,福興園川菜館,Foursquare API,http://4sq.com/ju7Its,NaN,NaN,NaN,[Chinese Restaurant],德行東路233-3號,146.643591
3,卡蜜拉早餐店,Foursquare API,http://4sq.com/14pig9E,NaN,NaN,NaN,[American Restaurant],NaN,146.586133
4,melody heart 旋律之心女僕咖啡館,Foursquare API,https://4sq.com/37ZCye2,NaN,NaN,NaN,[Café],重慶南路一段5巷1號,146.560261


## YELP API

In [17]:
import requests

In [18]:
yelp_base = 'https://api.yelp.com/v3/businesses/search'
yelp_header = {'Authorization': 'bearer ' + yelp_api_key} # hidden

### single query from multiple categories

In [19]:
# location
ll = '25.033964,121.564468'#25.0330° N, 121.5654° E TAIWAN 101 tower
# yelp categories: https://www.yelp.com/developers/documentation/v3/all_category_list
yelp_categories = {'restaurants':'restaurants','bars_general':'bars','landmarks':'landmarks'}
yelp_results = dict()

for yelp_cat_key,yelp_cat_code in yelp_categories.items():
    yelp_params = { 
                'category':yelp_cat_code,
                'limit':50,
                'radius': 25000,
                'location':ll}
    temp_res = requests.get(url=yelp_base, params=yelp_params, headers=yelp_header)

    yelp_results[yelp_cat_key] = temp_res.json()

    #print(f'status code: {temp_res.status_code}') 

In [20]:
print(type(yelp_results['restaurants']))
#yelp_json = yelp_results.json()
display = JSON(yelp_results)

<class 'dict'>


In [21]:
display

<IPython.core.display.JSON object>

In [22]:
def unpack_dict (dictio):
    """ 
    This how you transform a dict to a list but in a 
    feverish dream of unpacked values and lost mental sanity.
    works as long as the structure of the query is similar in depth"""
    res = list()
    for k1,v1 in dictio.items():
        for k2,v2 in  v1.items():
            
            try:
                for v3 in  v2:
                    res.append(v3)
            except TypeError as e:
                #there is the odd non-dictionary data that break the unpacking
                continue

    return res

In [23]:
test = unpack_dict(yelp_results)
display_m = JSON(test)
display_m

<IPython.core.display.JSON object>

In [24]:
# must modify unpack_dict to remove 'center values'
yelp_name = dict()

for i in range(len(test)):
    #filtering garbage data
    if isinstance(test[i],dict): 
        key = test[i]['name']
        yelp_name[key] = test[i]

    else:
        print('--not a dict--')
        
display= JSON(yelp_name)

--not a dict--
--not a dict--
--not a dict--


In [25]:
display

<IPython.core.display.JSON object>

In [26]:
yelp_name["Taipei 101"]['coordinates']

{'latitude': 25.0339855381681, 'longitude': 121.564472428561}

In [27]:
def yelp_venue_data_Panda(dictio):
    
    yelp_ready_data = {
    'name':[],
    'api':[],
    'api_url':[],
    'location_url':[], # doesn't have one
    'rating':[],
    'rating_count':[],
    'category':[],
    'location_address':[],
    'lat_lon':[]    
}
    
    for venue_name in dictio.keys():
        short_path = dictio[venue_name]
        # no need to check since it is our key 
        yelp_ready_data['name'].append(venue_name)
        yelp_ready_data['api'].append('yelpAPI')
        yelp_ready_data['api_url'].append(short_path.get('url',np.NaN))
        
        # yelp doesn't have the location's url website
        yelp_ready_data['location_url'].append(short_path.get('locationurl',np.NaN))# we keep it so that we get a NaN
        
        yelp_ready_data['rating'].append(short_path.get('rating',np.NaN)) # should work?
        
        yelp_ready_data['rating_count'].append(short_path.get('review_count',np.NaN))
        
        #categories
        categories_names = list()
        for i in range(len(short_path['categories'])):
            categories_names.append(short_path['categories'][i]['title'])
        yelp_ready_data['category'].append(categories_names)

        yelp_ready_data['location_address'].append(short_path['location'].get('address1',np.NaN))
        gps =  short_path['coordinates'].get('latitude',np.NaN) + short_path['coordinates'].get('longitude',np.NaN)
        # a single np.NaN voids any numbers present
        yelp_ready_data['lat_lon'].append(gps)
    return yelp_ready_data

In [28]:
y_r=yelp_venue_data_Panda(yelp_name)

In [50]:
yelp_SQL_DATA = pd.DataFrame(y_r)
yelp_SQL_DATA.head()

,name,api,api_url,location_url,rating,rating_count,category,location_address,lat_lon
0,Din Tai Fung,yelpAPI,https://www.yelp.com/biz/%E9%BC%8E%E6%B3%B0%E8...,NaN,4.5,63,"[Taiwanese, Shanghainese, Dim Sum]",南京西路12號,146.573407
1,Elephant Mountain Hiking Trail,yelpAPI,https://www.yelp.com/biz/%E8%B1%A1%E5%B1%B1%E8...,NaN,4.5,100,"[Hiking, Landmarks & Historical Buildings]",信義路五段150巷342弄,146.598162
2,Taipei 101,yelpAPI,https://www.yelp.com/biz/%E5%8F%B0%E5%8C%97101...,NaN,4.0,143,"[Landmarks & Historical Buildings, Shopping Ce...",信義路五段7號,146.598458
3,Addiction Aquatic Development,yelpAPI,https://www.yelp.com/biz/%E4%B8%8A%E5%BC%95%E6...,NaN,4.5,360,"[Seafood, Barbeque, Hot Pot]",民族東路410巷2弄18號,146.603781
4,Raohe Street Night Market,yelpAPI,https://www.yelp.com/biz/%E9%A5%92%E6%B2%B3%E8...,NaN,4.5,156,"[Street Vendors, Local Flavor, Shopping]",饒河街,146.628481


## SQL Data preparation

## pandas DataFrame columns filtering

In [30]:
physical_location_val=['name','location_address','location_url','lat_lon']

In [31]:
yelp_SQL_DATA[physical_location_val].head()

,name,location_address,location_url,lat_lon
0,Din Tai Fung,南京西路12號,NaN,146.573407
1,Elephant Mountain Hiking Trail,信義路五段150巷342弄,NaN,146.598162
2,Taipei 101,信義路五段7號,NaN,146.598458
3,Addiction Aquatic Development,民族東路410巷2弄18號,NaN,146.603781
4,Raohe Street Night Market,饒河街,NaN,146.628481


In [32]:
#f4sqr_SQL_DATA[physical_location_val].head() 

In [33]:
online_footprint_val=['api','api_url','rating','rating_count','name']

In [34]:
f4sqr_SQL_DATA[online_footprint_val].head() 

,api,api_url,rating,rating_count,name
0,Foursquare API,http://4sq.com/hS3Lgj,7.6,164.0,Taipei 101 Food Court (台北101美食街)
1,Foursquare API,http://4sq.com/9LUPOF,9.1,351.0,Din Tai Fung (鼎泰豐)
2,Foursquare API,http://4sq.com/ju7Its,NaN,NaN,福興園川菜館
3,Foursquare API,http://4sq.com/14pig9E,NaN,NaN,卡蜜拉早餐店
4,Foursquare API,https://4sq.com/37ZCye2,NaN,NaN,melody heart 旋律之心女僕咖啡館


In [35]:
#yelp_SQL_DATA[online_footprint_val].head()

### PANDAS SQL ready tables

In [36]:
physical_location_SQL_DATA = yelp_SQL_DATA[physical_location_val].append(f4sqr_SQL_DATA[physical_location_val])
physical_location_SQL_DATA.head()

,name,location_address,location_url,lat_lon
0,Din Tai Fung,南京西路12號,NaN,146.573407
1,Elephant Mountain Hiking Trail,信義路五段150巷342弄,NaN,146.598162
2,Taipei 101,信義路五段7號,NaN,146.598458
3,Addiction Aquatic Development,民族東路410巷2弄18號,NaN,146.603781
4,Raohe Street Night Market,饒河街,NaN,146.628481


In [37]:
online_footprint_SQL_DATA = yelp_SQL_DATA[online_footprint_val].append(f4sqr_SQL_DATA[online_footprint_val])
online_footprint_SQL_DATA.head()

,api,api_url,rating,rating_count,name
0,yelpAPI,https://www.yelp.com/biz/%E9%BC%8E%E6%B3%B0%E8...,4.5,63.0,Din Tai Fung
1,yelpAPI,https://www.yelp.com/biz/%E8%B1%A1%E5%B1%B1%E8...,4.5,100.0,Elephant Mountain Hiking Trail
2,yelpAPI,https://www.yelp.com/biz/%E5%8F%B0%E5%8C%97101...,4.0,143.0,Taipei 101
3,yelpAPI,https://www.yelp.com/biz/%E4%B8%8A%E5%BC%95%E6...,4.5,360.0,Addiction Aquatic Development
4,yelpAPI,https://www.yelp.com/biz/%E9%A5%92%E6%B2%B3%E8...,4.5,156.0,Raohe Street Night Market


## SQL tables creation

In [38]:
conn = sqlite3.connect('attractions.db')
c = conn.cursor()
connection = cf.create_connection("attractions.db") # in this case the database is in the same

Connection to SQLite DB successful


In [39]:
foreign_keep_enable='''PRAGMA foreign_keys = ON;'''
cf.ex_q(connection,foreign_keep_enable)

Query executed successfully


In [40]:
drop_physical_location_table = '''DROP TABLE IF EXISTS physical_location;'''

In [41]:
create_physical_location_table = """
CREATE TABLE IF NOT EXISTS physical_location(
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    location_address TEXT,
    location_url TEXT,
    lat_lon REAL
    );
"""

In [42]:
drop_online_footprint_table = '''DROP TABLE IF EXISTS online_footprint;'''

In [43]:
create_online_footprint_table = """
CREATE TABLE IF NOT EXISTS online_footprint(
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    api TEXT NOT NULL,
    api_url TEXT,
    rating REAL,
    rating_count INT,
    name TEXT NOT NULL, 
    FOREIGN KEY (name) REFERENCES physical_location (name)
    );
"""

In [44]:
cf.ex_q(connection,drop_physical_location_table)
cf.ex_q(connection,create_physical_location_table)
cf.ex_q(connection,drop_online_footprint_table)
cf.ex_q(connection,create_online_footprint_table)

Query executed successfully
Query executed successfully
Query executed successfully
Query executed successfully


### SQL DATA Export

In [45]:
physical_location_SQL_DATA.to_sql('physical_location', conn, if_exists='replace', index = False)
online_footprint_SQL_DATA.to_sql('online_footprint', conn, if_exists='replace', index = False)
#print('DataFrame insert successful')

In [46]:
# Populate the tables
try:
    physical_location_SQL_DATA.to_sql('physical_location', conn, if_exists='replace', index = False)
    print('DataFrame insert successful')
except:
    print('Something went wrong, are you connecteds to the database?')

DataFrame insert successful


## FROM QUERRIES TO INSANITY

In [47]:
pd.DataFrame(yelp_results)
#json_normalize(yelp_results).transpose()#,'restaurants')
#pd.DataFrame.from_records(yelp_results)

,restaurants,bars_general,landmarks
businesses,"[{'id': 'khfZV4LtCDDzlFoT-xMAkw', 'alias': '鼎泰...","[{'id': 'khfZV4LtCDDzlFoT-xMAkw', 'alias': '鼎泰...","[{'id': 'khfZV4LtCDDzlFoT-xMAkw', 'alias': '鼎泰..."
total,11300,11300,11300
region,"{'center': {'longitude': 121.56440734863281, '...","{'center': {'longitude': 121.56440734863281, '...","{'center': {'longitude': 121.56440734863281, '..."


In [48]:
help(dict.values)

Help on method_descriptor:

values(...)
    D.values() -> an object providing a view on D's values



I somewhat unpacked a full dict and my mind is blown

In [51]:
#init list
# evey it add one dict to list with keys for columsn and val for cells
def pd_normalized_list(t):
    for record in range(len(t)):

        if not isinstance(t[record], dict):
            #print(type(t[record]))
            pass
        elif record == 0:
            index = pd.DataFrame.from_dict(t[record], orient='index')
            #print(index)


        else:
            record_serie = pd.DataFrame.from_dict(t[record], orient='index')

            #if record in [1,2]:
            #    print(record_serie,'\n')

            heading = record_serie.loc['name'] 
            #print(heading[0])
            index[record] = record_serie
            #df_res = index.append(record_serie)
            #index = pd.concat([index,record_serie], ignore_index=True, axis=1)
    return index

#index.shape[0]
#index

In [54]:
t = unpack_dict(yelp_results)

print(type(t))


<class 'list'>


In [55]:
to = pd_normalized_list(unpack_dict(yelp_results))
to.head()

,0,1,2,3,4,5,6,7,8,9,...,142,143,144,145,146,147,148,149,150,151
id,khfZV4LtCDDzlFoT-xMAkw,HKR_FxnbRqGhhNtPmiTVmQ,LjB6ZK6OgRoWkRhVXCUJFA,aUwhNi7sYYOLfp2EKiEgmQ,Xbmdhb5PEfbAPNxzghWCRQ,jYL7G6_hHKLuVzgNKqVDOQ,i5QZlqsG51Xc7Dr9Ch3xdQ,5gNHAIyajmJ_4i5vXwOVRw,3Lhum3xy5RawYMtWpsD1fQ,NJkKxhVY3VY4ULM-XLrsaw,...,gJYlwKGZR27-Iyh5DiVwiQ,c-n2-s3SdKWn3PUfz9luKw,HA9_jLVRCv8WFVjo2Zk0_g,v1ln4Z3j0f65YaZjL4Y-xQ,Fe0UPyI5og76qRlJP4ZScw,qXvbofTJzyim1VYwCfsoWA,tskLWpQounhC65HgGgOFOA,XAxlHIZC9TxadGE7d0YaWg,bXS-ylyAWoiskqrtg8icAw,MgXBYxMhRZ8oV3dUmeYrjw
alias,鼎泰豐-信義區-2,鼎泰豐-大安區-3,象山親山步道-信義區,台北101-台北市信義區,上引水產-中山區,鼎泰豐-信義區,饒河街觀光夜市-松山區-2,永和豆漿大王-大安區-2,r-和-d-cocktail-lab-大安區,佳德糕餅-松山區,...,alchemy-信義區,慶城海南雞飯-台北市松山區,馬辣-信義區,拉拉熊咖啡廳-大安區,toasteria-cafe-大安區-6,松山文創園區-台北市信義區-2,饞食坊-大安區,喀漢堡-信義區,御品元傳統手工元宵-台北市大安區,indulge實踐創新餐酒館-大安區
name,Din Tai Fung,Din Tai Fung,Elephant Mountain Hiking Trail,Taipei 101,Addiction Aquatic Development,Din Tai Fung,Raohe Street Night Market,Yong He Soy Milk King,R&D Cocktail Lab,ChiaTe,...,Alchemy,Ching Cheng Hainan Chicken Rice,Mala,Rilakkuma Café,Toasteria Cafe,Songshan Cultural and Creative Park,Zen Food,Selfish Burger,御品元傳統手工元宵,Indulge實踐創新餐酒館
image_url,https://s3-media1.fl.yelpcdn.com/bphoto/jhpS2W...,https://s3-media1.fl.yelpcdn.com/bphoto/r2FEA8...,https://s3-media1.fl.yelpcdn.com/bphoto/LZ4P5E...,https://s3-media4.fl.yelpcdn.com/bphoto/l0lJAM...,https://s3-media4.fl.yelpcdn.com/bphoto/GPm6ZZ...,https://s3-media4.fl.yelpcdn.com/bphoto/Ff9xg7...,https://s3-media1.fl.yelpcdn.com/bphoto/8IB4RD...,https://s3-media1.fl.yelpcdn.com/bphoto/jUOyja...,https://s3-media3.fl.yelpcdn.com/bphoto/-mHuHp...,https://s3-media3.fl.yelpcdn.com/bphoto/uPPa3O...,...,https://s3-media4.fl.yelpcdn.com/bphoto/ALc-WU...,https://s3-media1.fl.yelpcdn.com/bphoto/hhhJ7H...,https://s3-media2.fl.yelpcdn.com/bphoto/_m9v7L...,https://s3-media3.fl.yelpcdn.com/bphoto/ppgNDu...,https://s3-media1.fl.yelpcdn.com/bphoto/ycFzUf...,https://s3-media1.fl.yelpcdn.com/bphoto/1V6v9R...,https://s3-media1.fl.yelpcdn.com/bphoto/VEfXYS...,https://s3-media2.fl.yelpcdn.com/bphoto/0VJ4bn...,https://s3-media3.fl.yelpcdn.com/bphoto/eva1iK...,https://s3-media1.fl.yelpcdn.com/bphoto/Trs5sw...
is_closed,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [56]:
tot = to.transpose()
type(tot)
tot

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,khfZV4LtCDDzlFoT-xMAkw,鼎泰豐-信義區-2,Din Tai Fung,https://s3-media1.fl.yelpcdn.com/bphoto/jhpS2W...,False,https://www.yelp.com/biz/%E9%BC%8E%E6%B3%B0%E8...,289,"[{'alias': 'shanghainese', 'title': 'Shanghain...",4.5,"{'latitude': 25.033337, 'longitude': 121.564307}",[],$$,"{'address1': '市府路45號', 'address2': 'B1樓', 'add...",+886281017799,+886 2 8101 7799,70.3687
1,HKR_FxnbRqGhhNtPmiTVmQ,鼎泰豐-大安區-3,Din Tai Fung,https://s3-media1.fl.yelpcdn.com/bphoto/r2FEA8...,False,https://www.yelp.com/biz/%E9%BC%8E%E6%B3%B0%E8...,347,"[{'alias': 'shanghainese', 'title': 'Shanghain...",4.5,"{'latitude': 25.033483, 'longitude': 121.530112}",[],$$,"{'address1': '信義路二段194號', 'address2': None, 'a...",+886223218928,+886 2 2321 8928,3455.64
2,LjB6ZK6OgRoWkRhVXCUJFA,象山親山步道-信義區,Elephant Mountain Hiking Trail,https://s3-media1.fl.yelpcdn.com/bphoto/LZ4P5E...,False,https://www.yelp.com/biz/%E8%B1%A1%E5%B1%B1%E8...,100,"[{'alias': 'hiking', 'title': 'Hiking'}, {'ali...",4.5,"{'latitude': 25.0273860627985, 'longitude': 12...",[],NaN,"{'address1': '信義路五段150巷342弄', 'address2': None...",,,972.953
3,aUwhNi7sYYOLfp2EKiEgmQ,台北101-台北市信義區,Taipei 101,https://s3-media4.fl.yelpcdn.com/bphoto/l0lJAM...,False,https://www.yelp.com/biz/%E5%8F%B0%E5%8C%97101...,143,"[{'alias': 'landmarks', 'title': 'Landmarks & ...",4,"{'latitude': 25.0339855381681, 'longitude': 12...",[],$$$,"{'address1': '信義路五段7號', 'address2': None, 'add...",+886281017777,+886 2 8101 7777,7.00844
4,Xbmdhb5PEfbAPNxzghWCRQ,上引水產-中山區,Addiction Aquatic Development,https://s3-media4.fl.yelpcdn.com/bphoto/GPm6ZZ...,False,https://www.yelp.com/biz/%E4%B8%8A%E5%BC%95%E6...,360,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...",4.5,"{'latitude': 25.066673, 'longitude': 121.537108}",[],$$,"{'address1': '民族東路410巷2弄18號', 'address2': None...",+886225081268,+886 2 2508 1268,4559.76
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147,qXvbofTJzyim1VYwCfsoWA,松山文創園區-台北市信義區-2,Songshan Cultural and Creative Park,https://s3-media1.fl.yelpcdn.com/bphoto/1V6v9R...,False,https://www.yelp.com/biz/%E6%9D%BE%E5%B1%B1%E6...,29,"[{'alias': 'culturalcenter', 'title': 'Cultura...",4,"{'latitude': 25.04392, 'longitude': 121.55794}",[],$,"{'address1': '光復南路133號', 'address2': '', 'addr...",+886227651388,+886 2 2765 1388,1194.95
148,tskLWpQounhC65HgGgOFOA,饞食坊-大安區,Zen Food,https://s3-media1.fl.yelpcdn.com/bphoto/VEfXYS...,False,https://www.yelp.com/biz/%E9%A5%9E%E9%A3%9F%E5...,27,"[{'alias': 'gastropubs', 'title': 'Gastropubs'}]",4.5,"{'latitude': 25.03265, 'longitude': 121.54402}",[],$$,"{'address1': '信義路四段30巷8弄13之1號', 'address2': ''...",+886227555859,+886 2 2755 5859,2057.01
149,XAxlHIZC9TxadGE7d0YaWg,喀漢堡-信義區,Selfish Burger,https://s3-media2.fl.yelpcdn.com/bphoto/0VJ4bn...,False,https://www.yelp.com/biz/%E5%96%80%E6%BC%A2%E5...,18,"[{'alias': 'burgers', 'title': 'Burgers'}]",4.5,"{'latitude': 25.03102, 'longitude': 121.56116}",[],$$,"{'address1': '莊敬路180 號', 'address2': '', 'addr...",+886223450825,+886 2 2345 0825,461.39
150,bXS-ylyAWoiskqrtg8icAw,御品元傳統手工元宵-台北市大安區,御品元傳統手工元宵,https://s3-media3.fl.yelpcdn.com/bphoto/eva1iK...,False,https://www.yelp.com/biz/%E5%BE%A1%E5%93%81%E5...,18,"[{'alias': 'streetvendors', 'title': 'Street V...",4.5,"{'latitude': 25.0302296, 'longitude': 121.5553...",[],$,"{'address1': '通化街39巷50弄31號', 'address2': '', '...",+88627009007,+886 2 700 9007,997.169
